# References

* Ned Batchelder: [Loop Like a Native]

[Loop Like a Native]: http://nedbatchelder.com/text/iter.html

# Recipe: Print List with Indices

Here is a straight-forward approach to print a list along with indices:

In [1]:
band = ['Peter', 'Paul', 'Mary']
for i in range(len(band)):
    print('{} - {}'.format(i, band[i]))

0 - Peter
1 - Paul
2 - Mary


There are several problems with this approach:

1. The expression in the for line is clunky at best
1. The look up expression, `band[i]` is not the most efficient
1. It is hard to understand

A more pythonic approach is to use the `enumerate` generator:

In [2]:
band = ['Peter', 'Paul', 'Mary']
for i, member in enumerate(band):
    print('{} - {}'.format(i, member))

0 - Peter
1 - Paul
2 - Mary


What if we want the index to start from a non-zero value? It turns out that enumerate can optionally take a second parameter for that:

In [3]:
for i, member in enumerate(band, 1):
    print('{} - {}'.format(i, member))

1 - Peter
2 - Paul
3 - Mary


# Looping through a Dictionary

By default, looping through a dictionary will give us a list of keys:

In [4]:
grades = {'John': 89, 'Anna': 91, 'Karen': 88}
for name in grades:
    print(name)

Karen
Anna
John


Many newbies use the following idiom to loop through both the keys and values of a dictionary:

In [5]:
grades = {'John': 89, 'Anna': 91, 'Karen': 88}
for name in grades:
    print('{}: {}'.format(name, grades[name]))

Karen: 88
Anna: 91
John: 89


This approach works, but it requires the look-up operations, which are not efficient: `grades[name]`. A more pythonic approach is:

In [6]:
grades = {'John': 89, 'Anna': 91, 'Karen': 88}
for name, grade in grades.items():
    print('{}: {}'.format(name, grade))

Karen: 88
Anna: 91
John: 89


# Looping through a Set, String


In [7]:
colors = {'red', 'green', 'blue'}
for color in colors:
    print(color)

print('---')
name = 'John'
for letter in name:
    print(letter)

blue
green
red
---
J
o
h
n


# Recipe: Loops through Two or More Lists

Here is the obvious, but non pythonic way:

In [8]:
user_ids = [501, 502, 503]
aliases = ['john', 'karen', 'anna']
for i in range(len(user_ids)):
    print('{} assigned to {}'.format(user_ids[i], aliases[i]))

501 assigned to john
502 assigned to karen
503 assigned to anna


Again, the problem with this approach is the need to look up the list and clumsy code. A better, more pythonic way is to use the zip function:

In [9]:
user_ids = [501, 502, 503]
aliases = ['john', 'karen', 'anna']
for user_id, alias in zip(user_ids, aliases):
    print('{} assigned to {}'.format(user_id, alias))

501 assigned to john
502 assigned to karen
503 assigned to anna


# The else Clause in Loop

Unlike other languages, the `for` and `while` loops in Python come with an `else` clause, which gets executed when the loop finished without any `break`. In this regard, the `else` clause should be called `nobreak`. One application of this feature is to find elements in a loop, if found we break out, if not, the `else` clause gets executed:

In [32]:
target = 8
my_list = [1, 3, 5, 7]

for index, item in enumerate(my_list):
    if item == target:
        print('{} found at index [{}]'.format(target, index))
        break
else:
    print('{} not found'.format(target))

8 not found


# Recipe: Find First Element in a Two-Dimensional Structure

Given a two dimentional structure such as one below:

In [10]:
spreadsheet = [
    [1, 2, 3, 4],     # Row 0
    [5, 6, 7, 8],     # Row 1
    [9, 10, 11, 12],  # Row 2
]

def print_matrix(matrix):
    print('Matrix:')
    for row in matrix:
        print(' '.join(str(cell) for cell in row))
    print('---')

How do we search for a value, and return the row and column where that value occurs? Given the two-dimensional nature of the problem, the most straight-forward approach is to use two nested loops:

In [11]:
target = 6

for row_number, row in enumerate(spreadsheet):
    print('Search row', row_number)
    for column_number, cell in enumerate(row):
        if cell == target:
            print('Found {} at row {}, column {}'.format(cell, row_number, column_number))
            break


Search row 0
Search row 1
Found 6 at row 1, column 1
Search row 2


The problem with the above is the break statement only break out of the inner loop, but not the outter one. In order to break out of both loops, we will need to put in some ugly hack which make the code unreadable. A better solution is to serialize all the cells in the 2D structure:

In [12]:
def get_cells(matrix):
    for row_number, row in enumerate(matrix):
        print('Search row', row_number)
        for column_number, cell in enumerate(row):
            yield cell, row_number, column_number

# Main
target = 7
for cell, row_number, column_number in get_cells(spreadsheet):
    if cell == target:
        print('Found {} at row {}, column {}'.format(cell, row_number, column_number))
        break
else:
    print('Failed to find', target)

Search row 0
Search row 1
Found 7 at row 1, column 2


Notes

* `get_cells` still need to have the nested loops, but it no longer contain any discriminating logic
* This solution admittedly used 3 loops, but the third one is not nested
* Not many people know that loops in Python (`while` and `for`) has an optional `else` clause. This clause is executed when the loop completed without any break statement. Raymond Hettinger said it should have been called `nobreak`

Now that we have implemented `get_cells`, we can make it more powerful by including a predicate for selecting cells, if not supplied, the default predicate will select all cells by return True for all cells:

In [13]:
def get_cells(matrix, predicate=lambda x: True):
    for row_number, row in enumerate(matrix):
        for column_number, cell in enumerate(row):
            if predicate(cell):
                yield cell, row_number, column_number
                
target = 7
for cell, row_number, column_number in get_cells(spreadsheet):
    if cell == target:
        print('Found {} at row {}, column {}'.format(cell, row_number, column_number))
        break


Found 7 at row 1, column 2


Here is an example use of that predicate. The following code will print out all the cells that are divisible by 3:

In [14]:
print_matrix(spreadsheet)
for cell_info in get_cells(spreadsheet, predicate=lambda x: x % 3 == 0):
    print('Found {} at row {}, column {}'.format(*cell_info))

Matrix:
1 2 3 4
5 6 7 8
9 10 11 12
---
Found 3 at row 0, column 2
Found 6 at row 1, column 1
Found 9 at row 2, column 0
Found 12 at row 2, column 3


# Accessing List Elements

Besides the usual indices such as 0, 1, 2... Python also allow negative indices:

In [15]:
mylist = 'zero one two three four five six seven eight nine ten'.split()
print('My list:', mylist)
print('First:', mylist[0])
print('Last:', mylist[-1])
print('Next to last:', mylist[-2])

My list: ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
First: zero
Last: ten
Next to last: nine


### Python also Allow Accessing Slices

In [16]:
print('First four:', mylist[:4])
print('Fifth element to last:', mylist[4:])
print('Fifth element up to, but not including, last:', mylist[4:-1])

First four: ['zero', 'one', 'two', 'three']
Fifth element to last: ['four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
Fifth element up to, but not including, last: ['four', 'five', 'six', 'seven', 'eight', 'nine']


Note:

* `mylist[:4]` is the same as `mylist[0:4]`
* `mylist[4:]` is the same as `mylist[4:None]`
* `mylist[:4] + mylist[4:] == mylist`

### Slices can Be named

In [17]:
record = (1, 5, 'John', 'Wayne', '555-1234', '555-2233')

COORDINATE = slice(0, 2)
NAME = slice(2, 4)
PHONES = slice(4, None)

print('Person:    ', record[NAME])
print('Coordinate:', record[COORDINATE])
print('Phones:    ', record[PHONES])

Person:     ('John', 'Wayne')
Coordinate: (1, 5)
Phones:     ('555-1234', '555-2233')


# Mutable vs. Immutable Collections

Lists, dictionaries and sets are mutable which means we can update, delete, insert elements or slices (list only). Tuple and frozenset are not mutable.

In [18]:
mylist = list(range(10))
print('Original:', mylist)

mylist[2] = 'deux'
print('Replaced element at index 2:', mylist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Replaced element at index 2: [0, 1, 'deux', 3, 4, 5, 6, 7, 8, 9]


In [19]:
mylist = list(range(10))
print('Original:', mylist)

del mylist[-2:]
print('Deleted last 2 element:', mylist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Deleted last 2 element: [0, 1, 2, 3, 4, 5, 6, 7]


In [20]:
mylist = list(range(10))
print('Original:', mylist)

del mylist[3:6]
print('Deleted elements at 3..5:', mylist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Deleted elements at 3..5: [0, 1, 2, 6, 7, 8, 9]


In [21]:
mylist = list(range(10))
print('Original:', mylist)

mylist[1:6] = ['une', 'deux', 'trois']
print('Replace elements 1..5:', mylist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Replace elements 1..5: [0, 'une', 'deux', 'trois', 6, 7, 8, 9]


# Recipe: Reverse a List

### First Approach: Using the Slice Notation

A slice notation can take yet a third, optional number to control the stepping whereas 1 means all elements, 2 means every other element, and -1 means stepping backward. This is the fastest method, but also a little terse.

In [22]:
mylist = list(range(10))
print('Original:', mylist)

mylist = mylist[::-1]
print('Reversed:', mylist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Reversed: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


### Second Approach: Using the `reverse` Method

The list object comes with a `reverse` method which will alter the list, just like the previous way. This method is the recommended approach for being easy to understand at the cost of a little (probably small) performance hit compare to the first approach.

In [23]:
mylist = list(range(10))
print('Original:', mylist)

mylist.reverse()
print('Reversed:', mylist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Reversed: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


### Third Approach: Keep the Original and Create a Reversed Copy

Another way to reverse a list is to pass it into the reversed function. Note that this function does not alter the list, but rather return a reversed list:

In [24]:
mylist = list(range(10))
print('Original:', mylist)

newlist = list(reversed(mylist))
print('After reversing')
print('Original:', mylist)
print('New list:', newlist)

Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
After reversing
Original: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
New list: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
